# Vizualizando os pontos de prevalência de vítimas de crimes no território brasileiro

In [41]:
#importing modules
import pandas as pd
import numpy as np
import folium
from folium import plugins

In [42]:
#import dataset
crimes_file = "crimes-dataset.tsv"
crimes_df = pd.read_csv(crimes_file, sep="\t")


In [43]:
#vizualizando dataset
crimes_df.head()

,codigo_ibge,uf,nome,latitude,longitude,data,vitimas
0,1200708,AC,Xapuri,-10.65160,-68.4969,2020-12-01,0
1,1200609,AC,Tarauacá,-8.15697,-70.7722,2020-12-01,0
2,1200450,AC,Senador Guiomard,-10.14970,-67.7362,2020-12-01,0
3,1200500,AC,Sena Madureira,-9.06596,-68.6571,2020-12-01,0
4,1200435,AC,Santa Rosa do Purus,-9.44652,-70.4902,2020-12-01,0


In [44]:
#Como a tarefa é vizualizar os crimes por mês, vou desconsiderar os dias, 
#apagando eles da string data de cada linha do dataset
def desconsiderar_dia(row):
  row.data = row.data[:-3]
  return row

crimes_df = crimes_df.apply(desconsiderar_dia, axis='columns')

In [45]:
#Dessa maneira, vou excluir esses municípios que não registraram data,
#que nesse dataset tinham o valor da data preenchido com 0s
crimes_df = crimes_df[crimes_df.data != '0000-00']
#E também os municípios que não registraram vitimas, caso contrário eles
#também seriam plotados no nosso mapa
crimes_df = crimes_df[crimes_df.vitimas>0]
#E por fim, ordenar o dataframe pela data
crimes_df = crimes_df.sort_values(by="data")

In [46]:
crimes_df.head()

,codigo_ibge,uf,nome,latitude,longitude,data,vitimas
179268,2300101,CE,Abaiara,-7.34588,-39.0416,2018-01,1
111565,4218707,SC,Tubarão,-28.47130,-49.0144,2018-01,1
111570,4218301,SC,Três Barras,-26.10560,-50.3197,2018-01,1
111574,4218004,SC,Tijucas,-27.23540,-48.6322,2018-01,1
111593,4216602,SC,São José,-27.61360,-48.6366,2018-01,4


In [47]:
#vou normalizar o número de vítimas para um valor entre 0 e 1
valor_max = crimes_df.vitimas.max()
crimes_df.vitimas = crimes_df.vitimas/valor_max

In [48]:
#Agora vamos definir uma função que precisaremos usar mais a diante
def df_to_useful_info(df):
  """
    input: DataFrame
    A partir de um DataFrame retorna uma lista 
    de latitude and longitude and e o peso
    dessa informação de acordo com o número de
    vítimas.
    output: list
  """
  coordenadas=[]
  for lat,lng,peso in zip(df.latitude.values,df.longitude.values,df.vitimas.values):
    coordenadas.append([lat,lng,peso])
  return coordenadas

In [49]:
#Agora, como queremos fazer um mapa que varia no tempo,
#temos que fazer uma lista de listas com a informação
#que vamos dar ao Folium para cada periodo de tempo.

#Nesse caso, estamos trabalhando com o tempo em meses
#e vamos usar nossa função suporte para extrair a 
#informação que queremos de cada mês.
all_data = []
for e in crimes_df.data.unique():
  temp = crimes_df[crimes_df.data == e]
  useful_data = df_to_useful_info(temp)
  all_data.append(useful_data)

In [50]:
#criando uma lista index com os meses que aparecerão na legenda
lista_index = list(crimes_df.data.unique())
lista_index.sort()

In [51]:
# Renderizando o mapa com algumas personalizações:
try:
  del(mapa)
except:
    pass
finally:
    mapa = folium.Map(location=[-15.788497,-47.879873],zoom_start=5,tiles='Stamen Toner')

In [52]:
# Adicionando os registros no mapa de calor:
hm = plugins.HeatMapWithTime(all_data, index=lista_index, radius=25, min_opacity = 0, max_opacity=0.8, auto_play=True)
hm.add_to(mapa)
mapa

In [24]:
#Salvando o arquivo
mapa.save('mapa-calor-cronologico-crimes.html')